In [1]:
import certifi
import pandas as pd
import json
import urllib3
from urllib3 import request
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px
from datetime import date
today = date.today()


In [5]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
state = 'NY'
date = '2007-01-01'
disaster = 'Snow'
api_address="https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$filter=incidentType%20eq%20%27"+disaster+"%27%20and%20incidentBeginDate%20gt%20%27"+date+"T04:00:00.000z%27%20and%20state%20eq%20%27"+state+"%27"
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
r = http.request('GET', api_address)
r.status

200

In [18]:
# for the state of Wyoming - WY from 2019-01-01 -> No data is Present
#Change it to Alabama - AL from 2019-01-01 to view data
data = json.loads(r.data.decode('utf-8'))
if data['DisasterDeclarationsSummaries'] == []:
        error = "No Data Avaliable in this Time Period for the state "+state
        print(error)    
else:
    df = pd.json_normalize(data, 'DisasterDeclarationsSummaries')
    df['fips'] = df['fipsStateCode']+df['fipsCountyCode']
    df['values'] = 1
    df['incidentdate'] = df.declarationDate.str[:10]
    # for date in df['incidentdate']:
    #     if today == date:
    #         print("Send Message")
    #     else:
    #         print("No Message")

In [22]:
fig = px.choropleth_mapbox(df, geojson=counties, locations='fips',
                            color_continuous_scale="Viridis",
                            range_color=(0, 12),
                            mapbox_style="open-street-map",
                            zoom=3.5, center = {"lat": 37.0902, "lon": -95.7129},
                            opacity=0.5,
                            hover_name = df['designatedArea'],
                            hover_data=["incidentType", "incidentdate", "declarationTitle"]
                            )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()